In [ ]:
# Initialization

Version = '01'

Description = """
    Script used for performing the indentation analysis
    during the BME Labs

    Version Control:
        01 - Original script

    Author: Mathieu Simon
            ARTORG Center for Biomedical Engineering Research
            SITEM Insel, University of Bern

    Date: February 2024
    """

In [ ]:
# Modules import

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
# Define functions

def ComputeEpsilon(m):

    mValues = [1.0, 1.1, 1.2, 1.3, 1.4, 1.5,
                1.6, 1.7, 1.8, 1.9, 2.0]
    EValues = [1.000, 0.829, 0.793, 0.772, 0.759, 0.750,
                0.743, 0.738, 0.733, 0.730, 0.727]

    Round = round(m-0.05,1)
    Idx = mValues.index(Round)
    Delta = m - Round
    Epsilon = EValues[Idx] + Delta * (EValues[Idx+1] - EValues[Idx])

    return Epsilon

def ProjectedArea(hc):

    """
    Compute projected contact area for Berkovich indenter
    Formula from the machine
    """

    if hc > 165:
        print("hc out of tip calibration !")

    Ap = 24.5 * hc**2
    Ap -= 860 * hc
    Ap += 2.01E005 * hc**(1/2)
    Ap -= 2.05E006 * hc**(1/4)
    Ap += 5.13E006 * hc**(1/8)
    Ap -= 3.30E006 * hc**(1/16)

    return Ap


In [ ]:
# Read file
CWD = Path.cwd()
Files = [F for F in CWD.iterdir() if F.name.endswith('Curves.TXT')]
Curves = pd.read_csv(Files[0], sep='\t')

# Select columns
Columns = Curves.columns
Curves = Curves[Columns[[0,1,3]]]
Curves.columns = ['Time (s)', 'Force (mN)','Displacement (nm)']

# Plot results
Figure, Axis = plt.subplots(1,2,dpi=196, figsize=(10,4))
Axis2 = Axis[0].twinx()
Axis[0].plot(Curves['Time (s)'], Curves['Force (mN)'],
            color=(0,0,1), label='Force (mN)')
Axis2.plot(Curves['Time (s)'], Curves['Displacement (nm)'],
            color=(1,0,0), label='Displacement (nm)')
Axis[0].set_xlabel('Time (s)')
Axis[0].set_ylabel('Force (mN)', color=(0,0,1))
Axis2.set_ylabel('Displacement (nm)', color=(1,0,0))
Axis[1].plot(Curves['Displacement (nm)'], Curves['Force (mN)'], color=(0,0,0))
Axis[1].set_xlabel('Displacement (nm)')
Axis[1].set_ylabel('Force (mN)')
Figure.tight_layout()
plt.show(Figure)

In [ ]:
# Select data for curve fitting
Idx = Curves['Displacement (nm)'].idxmax()
FMax = Curves.loc[Idx,'Force (mN)']
hMax = Curves.loc[Idx,'Displacement (nm)']
F1 = Curves.iloc[Idx:]['Force (mN)'] < 0.98 * FMax
F2 = Curves.iloc[Idx:]['Force (mN)'] > 0.40 * FMax
X = Curves.iloc[Idx:][F1*F2]['Displacement (nm)']
Y = Curves.iloc[Idx:][F1*F2]['Force (mN)']

# Fit curve on data and plot results
def OliverAndPharrGuess(h, hp, m):
    return FMax * ((h - hp) / (hMax - hp))**m

[hp, m], Pcov = curve_fit(OliverAndPharrGuess, X, Y)

def OliverAndPharr(h, hmax, hp, m):
    return FMax * ((h - hp) / (hmax - hp))**m

[hmax, hp, m], Pcov = curve_fit(OliverAndPharr, X, Y, p0=(hMax, hp, m))

Figure, Axis = plt.subplots(1,2,dpi=196,figsize=(10,4))
Axis[0].plot(Curves['Displacement (nm)'], Curves['Force (mN)'], color=(0,0,0),label='Data')
Axis[0].plot(X, OliverAndPharr(X, hmax, hp, m), color=(1,0,0), label='Fit')
Axis[1].plot(X, Y, color=(0,0,0), marker='o', linestyle='none', fillstyle='none')
Axis[1].plot(X, OliverAndPharr(X, hmax, hp, m), color=(1,0,0))
for i in range(2):
    Axis[i].set_xlabel('Displacement (nm)')
    Axis[i].set_ylabel('Force (mN)')
Figure.legend(loc='upper center', bbox_to_anchor=(0.5, 1.0), ncol=2)
plt.show(Figure)

In [ ]:
# Compute values 
Stiffness = m * FMax / (hmax - hp)
hr = hmax - FMax / Stiffness
Epsilon = ComputeEpsilon(m)
hc = hmax - Epsilon * (hmax - hr)

# User defined properties
Ap = ProjectedArea(hc)
Beta = 1.034
nu_i = 0.07  # Indenter Poissons' coefficient
Ei = 1141    # Intenter elastic modulus
nu_s = 0.3   # Sample estimated Poissons' coefficient

# Moduli
Er = (np.sqrt(np.pi) * Stiffness) / (2 * Beta * np.sqrt(Ap))
Er = Er * 10**6     # Convert to GPa
Es = 1 / (1/Er - (1 - nu_i**2)/Ei)
E = Es * (1-nu_s**2)